# Testing code for OLU project UROP Spring of 2017

Aditya Pidaparti, Arindam Banerjee, Nicholas Johnson

## Data Setup
The following cell contains all code that will be become datasetup.py.

In [27]:
from pandas_datareader.data import DataReader as web #irrelevant
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from datasets import *
from sparse_port_admm import sparse_port_admm as spa

## Lazy Updates ADMM
The following cells contains the new lazy_updates_admm.py code

This is our main function wrapper. It's going to perform setup for each test we do. That is, once we run this function, our initial paramters have been inputted and will stay that way (i.e. the learning rate and the L1 norm). It's going to come in the form of a function return results in the form of a pandas dataframe and a dictionary with all the parameters

### Parameters we'll be varying:
* learning rate (eta)
* transaction cost (gamma)
* risk tolerance (maxRisk)


In [30]:
def lazy_updates_admm(pricesData='../Data/nyse-o.csv',betasData='../Data/nyse-o_betas_30.csv', eta = 0.05, alpha = 0.01, gamma = 0.0 ,maxRisk = 10 ** 10, debug = True):
    
    #pricesData is the price relative matrix, a string to its location
    #betasData is the beta dataset, a string to its location
    #eta is learning rate
    #alpha is L1 norm weight (not varied)
    #gamma is transaction cost
    #maxRisk is max portfolio risk
    #if debug is True, print statements will appear
    
    r = .01 #Augmentation term
    b = 2 #weight on L2 norm
    
    #Data setup
    prices = pd.read_csv(pricesData,delimiter=',').as_matrix()
    betas = pd.read_csv(betasData,delimiter=',',index_col=0).as_matrix()
    
    #Variable initialization
    (num_stock, num_days) = prices.shape
    weight = np.zeros(prices.shape)
    wealth = np.zeros((num_days, 1))
    wealth[0,0] = 1 # Start with 1 dollar on day 1
    
    #portfolio for 1st day, allocate wealth equally across assets
    weight[0:, 0] = np.ones(num_stock) * (1/num_stock)
    
    #2nd through last day
    for day in range(1, num_days):

        prev_day_weight = weight[:, day-1]
        prev_day_data = prices[:, day-1]

        w = spa(prev_day_weight, prev_day_data, eta, b, alpha, r,debug=True)
       
        weight[:, day] = w
        new_wealth = wealth[day-1, 0] * np.dot(weight[:, day], prices[:, day])
        
        #Apply transaction cost. Zero for a gamma of 0
        transaction_cost = gamma*abs(wealth[day-1, 0])*np.linalg.norm(weight[:,day-1]-weight[:,day],1)
        wealth[day, 0] = new_wealth - transaction_cost

    plt.plot(wealth[0:day])
    plt.show()
    return [eta, alpha, gamma, rho, beta, wealth[num_days -1, 0]]

In [31]:
lazy_updates_admm()

---- ITER 1 ----
w: [  4.56651444e-04   8.62061605e-05   6.95327599e-04 ...,   0.00000000e+00
   0.00000000e+00   2.47967747e-05]
z: [ 0.  0.  0. ...,  0.  0.  0.]
u: [  2.79660293e-04  -9.07849899e-05   5.18336449e-04 ...,  -1.76991150e-04
  -1.76991150e-04  -1.52194376e-04]
0.0224847448903 0.0133038021048 0.0
0.0181265717205 0.00774149563809 0.0 0.00751846084636
---- ITER 2 ----
w: [  4.55777196e-04   8.71749238e-05   6.93265908e-04 ...,   0.00000000e+00
   0.00000000e+00   2.60710573e-05]
z: [ 0.  0.  0. ...,  0.  0.  0.]
u: [ 0.00055845 -0.0001806   0.00103461 ..., -0.00035398 -0.00035398
 -0.00030311]
0.0224351091787 0.0133038021048 0.0
0.0180649653588 0.00774099928097 0.0 0.00752026734046
---- ITER 3 ----
w: [  4.54901000e-04   8.81325699e-05   6.91208176e-04 ...,   0.00000000e+00
   0.00000000e+00   2.73327027e-05]
z: [ 0.  0.  0. ...,  0.  0.  0.]
u: [ 0.00083636 -0.00026946  0.00154883 ..., -0.00053097 -0.00053097
 -0.00045277]
0.0223855770027 0.0133038021048 0.0
0.01800341376

KeyboardInterrupt: 